Librerias

In [52]:
#LIBRERIAS DEL PROYECTO

from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
import os

#Este es driver que va a navegar.
driver = webdriver.Chrome()

Iniciar sesión

In [53]:
# Función para iniciar sesión
arancia_url = 'http://arancia-001-site1.btempurl.com/'

driver.get(arancia_url)
time.sleep(3)

# Entrar a la Web
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# Colocar las claves
element = driver.find_element(By.ID, 'TextBox1')
element.send_keys('FRANCESC')
element = driver.find_element(By.ID, 'TextBox2')
element.send_keys('n4r4nj4720')

# click en entrar
element = driver.find_element(By.ID,"Button1")
element.click()
time.sleep(3)

# clickear en la pestaña de "facturación"
element = driver.find_element(By.ID,"Button10")
element.click()
time.sleep(3)

# clickear en la sub-pestaña de "facturación" (para cargar facturas de ventas)
driver.switch_to.frame('facturacion')
element = driver.find_element(By.ID,"Button5")
element.click()
time.sleep(1)

Cambiar de frame al marco interno

In [54]:
driver.switch_to.frame('marco')

Establecer factura manual

In [55]:
# Quitar check de "aéreo"
element = driver.find_element(By.ID,"CheckBoxList1_0")
element.click()
time.sleep(2)

# Poner check en "manual"
element = driver.find_element(By.ID,"CheckBoxList1_3")
element.click()
time.sleep(2)

# Buscar el input de IVAS
element = driver.find_element(By.ID, "contar")

# Seleccionar todo el texto actual
element.send_keys(Keys.CONTROL, "a")
time.sleep(0.2)

# Escribir el nuevo valor (3 por la información que manejamos)
element.send_keys("3")

# Disparar el evento de cambio usando TAB (como si el usuario saliera del campo)
element.send_keys(Keys.TAB)
time.sleep(2)

CAMBIAR LOS IVAS

In [57]:
# Valores específicos para cada campo
valores = {
    "ivasa1": "0",
    "ivasa2": "10.5"  # con punto, ojo a la compa que rompe la página
}

for field_id, valor in valores.items():
    element = driver.find_element(By.ID, field_id)

    # Seleccionar todo el texto actual
    for i in range(4):
        element.send_keys(Keys.BACKSPACE)
    time.sleep(1)

    # Escribir directamente el nuevo valor (sin dejar el campo vacío)
    element.send_keys(valor)
    time.sleep(3)

Hasta aquí logramos que el sistema configure todo para empezar a cargar facturas....
Para que funcione, creo que debemos hacer lo siguiente:
- pasarle la lista de facturas que queremos que cargue + el csv de lo que ya existe
- que haga una comparación con las facturas que existen actualmente
- que las cargue. 

Para una primera versión, podemos hacer la comparación manual tal vez.

PASO FALTANTE: PROCESAR FACTURAS PARA OBTENER DATOS

In [58]:
data = pd.read_excel(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Emitidos - CUIT 30714894346.xlsx', header=0, skiprows=1)
data['Tipo'].unique()

array(['1 - Factura A',
       '201 - Factura de Crédito electrónica MiPyMEs (FCE) A',
       '203 - Nota de Crédito electrónica MiPyMEs (FCE) A',
       '6 - Factura B', '195 - Factura T'], dtype=object)

In [59]:
import pandas as pd
import string
import numpy as np
import re

data = pd.read_excel(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Emitidos - CUIT 30714894346.xlsx', header=0, skiprows=1)

#TIPO DE FACTURA
data['codigo_fc'] = [int(re.match(r'(\d+)', x).group(1)) for x in data['Tipo']]
data['Tipo'] = data['Tipo'].astype('string').str[-9:]
data['Tipo2'] = data['Tipo'].str[:7]
data['Tipo3'] = data['Tipo'].str[-1:]

data['tipo2_new'] = np.where(data['codigo_fc'].isin([1,6,11]), "Factura", "Credito")
data['tipo2_new'] = np.where(data['codigo_fc'] == 201, "Pyme_fc", data['tipo2_new'])
data['tipo2_new'] = np.where(data['codigo_fc'] == 203, "Pyme_nc", data['tipo2_new'])

data['tipo3_new'] = np.where(data['codigo_fc'].isin([1,3,201,203]), "A", "C")
data['tipo3_new'] = np.where(data['codigo_fc'].isin([6,8]), "B", data['tipo3_new'])

data['tipo3_new']

# FACTURA
data['Punto de Venta'] = data['Punto de Venta'].astype(str).str.zfill(5)
data['Número Desde'] = data['Número Desde'].astype(str).str.zfill(8)
data['Factura'] = data['Número Desde'].astype(int)

# Cliente
data['Denominación Receptor'] = data['Denominación Receptor'].str.translate(
    str.maketrans('', '', string.punctuation))
data['Cliente'] = data['Denominación Receptor'].str[:35]

# CUIT
data['CUIT'] = data['Nro. Doc. Receptor'].astype(str)


# IMPORTES
data['NETO 10.5'] = data['Neto Grav. IVA 10,5%'] 
data['IVA 10.5'] = data['IVA 10,5%'] 
data['NETO 21'] = data['Neto Grav. IVA 21%'] 
data['IVA 21'] = data['IVA 21%'] 
data['NO GRAVADO'] = data['Neto No Gravado'] 
data['EXENTO'] = data['Op. Exentas'] 
data['IMPUESTOS'] = data['Otros Tributos']
data['NETO 0'] = data['Neto Grav. IVA 0%']
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])
data['TOTAL_10.5'] = round(data['NETO 10.5'] + data['IVA 10.5'],2)
data['TOTAL_21'] = round(data['NETO 21'] + data['IVA 21'],2)


# PONER 0 A TODAS LAS COLUMNAS QUE NO TIENEN ALGO
columnas = ['NETO 0', 'NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS',
            'IVA 2,5%', 'IVA 5%', 'IVA 27%', 'TOTAL_NO_GRAVADO', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas: 
    data[col] = data[col].fillna(0)

# TIPO DE CAMBIO ESTE CAMBIAR PORQUE CAMBIA A FUTURO
columnas_tc = ['NETO 0','NETO 10.5', 'IVA 10.5', 'NETO 21', 'IVA 21', 'NO GRAVADO', 'EXENTO', 'IMPUESTOS', 'TOTAL_10.5', 'TOTAL_21'] 
for col in columnas_tc: 
    data[col] = data[col] * data['Tipo Cambio']

# SUMAR LO QUE NO LLEVA IVA
data['TOTAL_NO_GRAVADO'] = (data['NO GRAVADO'] + data['EXENTO'] + data['IMPUESTOS'] + data['NETO 0'])

# Convertimos las columnas a str
data['NETO 10.5'] = data['NETO 10.5'].astype(str)
data['IVA 10.5'] = data['IVA 10.5'].astype(str)
data['NETO 21'] = data['NETO 21'].astype(str)
data['IVA 21'] = data['IVA 21'].astype(str)
data['TOTAL_21'] = data['TOTAL_21'].astype(str)
data['TOTAL_10.5'] = data['TOTAL_10.5'].astype(str)

# PONER EN NEGATIVO LAS COLUMNAS QUE SON CREDITO
for col in columnas:
    mask = (data['tipo2_new'] == 'Credito') | (data['tipo2_new'] == 'Pyme_nc')
    data.loc[mask, col] = -data.loc[mask, col].astype(float)
    data[col] = data[col].astype(str)

# CREAR Y APLICAR MÁSCARA
clean = ['Fecha','tipo2_new','tipo3_new', 'Factura',
        'Cliente','CUIT', 'TOTAL_10.5', 'TOTAL_21',
        'TOTAL_NO_GRAVADO']

clean_data = data[clean]
clean_data

,Fecha,tipo2_new,tipo3_new,Factura,Cliente,CUIT,TOTAL_10.5,TOTAL_21,TOTAL_NO_GRAVADO
0,01/10/2025,Factura,A,3237,PILISAR S A,33709180849,379961.2,30681.22,7158.85
1,01/10/2025,Factura,A,3236,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,70149.06,885106.0
2,01/10/2025,Pyme_fc,A,213,PILISAR S A,33709180849,0.0,911434.46,11500025.0
3,01/10/2025,Pyme_fc,A,212,NEWSAN SOCIEDAD ANONIMA,30642617555,0.0,911434.46,11500025.0
4,01/10/2025,Pyme_nc,A,16,NEWSAN SOCIEDAD ANONIMA,30642617555,-0.0,-911434.46,-11500025.0
...,...,...,...,...,...,...,...,...,...
59,17/10/2025,Factura,A,3270,NEW PGA S R L,30615265450,1094471.24,519215.38,14317.75
60,17/10/2025,Factura,B,3601,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,1089728.0
61,17/10/2025,Factura,B,3602,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,1029465.6
62,17/10/2025,Factura,B,3603,LA IGLESIA UNIVERSAL DEL REINO DE D,33648976289,0.0,0.0,238530.56


Hemos logrado crear un dataset con todas las facturas que tiene el sistema, ahora solo falta que las cargue.


De aquí hacia abajo no funciona la cosa

----------------------



In [66]:
for index, row in clean_data.iterrows():
    # TOTAL_NO_GRAVADO
    elem = driver.find_element(By.ID, 'total1')
    elem.send_keys(Keys.CONTROL, 'a')  # Selecciona todo
    elem.send_keys(row['TOTAL_NO_GRAVADO'])  # Sobreescribe
    elem.send_keys(Keys.TAB)
    time.sleep(4)

    # TOTAL_10.5
    elem = driver.find_element(By.ID, 'total2')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_10.5'])
    elem.send_keys(Keys.TAB)
    time.sleep(4)

    # TOTAL_21
    elem = driver.find_element(By.NAME, 'total3')
    elem.send_keys(Keys.CONTROL, 'a')
    elem.send_keys(row['TOTAL_21'])
    elem.send_keys(Keys.TAB)
    time.sleep(4)

    driver.find_element(By.ID, "Button2").click()
    time.sleep(3)

    

KeyboardInterrupt: 

In [60]:
for index, row in clean_data.iterrows():
    # Encuentra cada campo por su id y llena el valor
    driver.find_element(By.ID, 'total1').send_keys(row['TOTAL_NO_GRAVADO'])
    time.sleep(1)
    driver.find_element(By.ID, 'total2').send_keys(row['TOTAL_10.5'])
    time.sleep(1)
    driver.find_element(By.NAME, 'total3').send_keys(row['TOTAL_21'])
    time.sleep(1)
    # driver.find_element(By.NAME, 'DetailsView1$ctl03').send_keys(row['Factura'])
    # driver.find_element(By.NAME, 'DetailsView1$ctl04').send_keys(row['Proveedor'])
    # driver.find_element(By.NAME, 'DetailsView1$ctl05').send_keys(row['CUIT'])
    # driver.find_element(By.NAME, 'DetailsView1$TextBox3').send_keys(row['NETO 10.5'])
    # driver.find_element(By.NAME, 'DetailsView1$TextBox4').send_keys(row['NETO 21'])
    # driver.find_element(By.NAME, 'DetailsView1$TextBox5').send_keys(row['IVA 10.5'])
    # driver.find_element(By.NAME, 'DetailsView1$TextBox6').send_keys(row['IVA 21'])
    # driver.find_element(By.NAME, 'DetailsView1$TextBox7').send_keys(row['TOTAL_NO_GRAVADO'])
    # # Haz clic en el botón "Agregar"
    # driver.find_element(By.LINK_TEXT, 'Agregar').click()
    # # Espera un poco para que la página se actualice
    # time.sleep(3)

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"[id="total1"]"}
  (Session info: chrome=141.0.7390.108); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x0x7ff78475e9e5+80021]
	GetHandleVerifier [0x0x7ff78475ea40+80112]
	(No symbol) [0x0x7ff7844e060f]
	(No symbol) [0x0x7ff784538854]
	(No symbol) [0x0x7ff784538b1c]
	(No symbol) [0x0x7ff78458c927]
	(No symbol) [0x0x7ff78456126f]
	(No symbol) [0x0x7ff78458968a]
	(No symbol) [0x0x7ff784561003]
	(No symbol) [0x0x7ff7845295d1]
	(No symbol) [0x0x7ff78452a3f3]
	GetHandleVerifier [0x0x7ff784a1dd8d+2960445]
	GetHandleVerifier [0x0x7ff784a1804a+2936570]
	GetHandleVerifier [0x0x7ff784a38a87+3070263]
	GetHandleVerifier [0x0x7ff7847784ce+185214]
	GetHandleVerifier [0x0x7ff78477ff1f+216527]
	GetHandleVerifier [0x0x7ff784767c24+117460]
	GetHandleVerifier [0x0x

In [20]:

for index, row in clean_data.iterrows():
    # Encuentra cada campo por su id y llena el valor
    driver.find_element(By.ID, 'DetailsView1_TextBox1').send_keys(row['Fecha'])
    driver.find_element(By.ID, 'DetailsView1_TextBox2').send_keys(row['Fecha'])
    driver.find_element(By.NAME, 'DetailsView1$ctl02').send_keys(row['Tipo3'])
    driver.find_element(By.NAME, 'DetailsView1$ctl03').send_keys(row['Factura'])
    driver.find_element(By.NAME, 'DetailsView1$ctl04').send_keys(row['Proveedor'])
    driver.find_element(By.NAME, 'DetailsView1$ctl05').send_keys(row['CUIT'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox3').send_keys(row['NETO 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox4').send_keys(row['NETO 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox5').send_keys(row['IVA 10.5'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox6').send_keys(row['IVA 21'])
    driver.find_element(By.NAME, 'DetailsView1$TextBox7').send_keys(row['TOTAL_NO_GRAVADO'])
    # Haz clic en el botón "Agregar"
    driver.find_element(By.LINK_TEXT, 'Agregar').click()
    # Espera un poco para que la página se actualice
    time.sleep(3)

In [21]:
os.remove(r'C:\Users\ThinkPad-PC\Downloads\Mis Comprobantes Recibidos - CUIT 30714894346.xlsx')